In [73]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score
from training_code import *
from load_data import initialize_test
from reading_datasets import read_test
from labels_to_ids import task5_labels_to_ids
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [74]:
def main(model_load_location):
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../2022.07.07_task5/'
    test_data = read_test(dataset_location , split = 'dev')

    labels_to_ids = task5_labels_to_ids
    input_data = (test_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time

    tokenizer = AutoTokenizer.from_pretrained(model_load_location)
    model = AutoModelForSequenceClassification.from_pretrained(model_load_location)

    # unshuffled testing data
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    # Getting testing dataloaders
    test_loader = initialize_test(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = False)

    test_ind_f1 = 0
    test_ind_precision = 0
    test_ind_recall = 0

    start = time.time()

    # Run the model with unshuffled testing data
    test_result = testing(model, test_loader, labels_to_ids, device)

    now = time.time()

    print('TIME TO COMPLETE:', (now-start)/60 )
    print()

    return test_result

In [75]:
dev_data = pd.read_table("dev.tsv")

In [76]:
dev_data

,tweet_id,tweet_text,label
0,100460,Esta farmacéutica garantiza la producción de 2...,Lit-News_mentions
1,301908,Dormir con la ventana abierta porque hace calo...,non-personal_reports
2,300889,@pdhernandezf @roa_flores Los test de coronavi...,non-personal_reports
3,101501,La Universidad de Buenos Aires entrena perros ...,Lit-News_mentions
4,105013,Mi amiga Nancy tiene #Coronavirus està aislada...,Lit-News_mentions
...,...,...,...
2006,301284,"Al momento de su secuestro, el diputado @TonyG...",non-personal_reports
2007,100251,Finlandia recurre a perros para detectar el Co...,Lit-News_mentions
2008,103444,📢 Si presentas algunos de estos síntomas: male...,Lit-News_mentions
2009,104896,Si presenta síntomas propios del #Coronavirus ...,Lit-News_mentions


In [77]:
if __name__ == '__main__':
    
    models = ['bert-base-multilingual-uncased0', 'bert-base-multilingual-uncased_oversampled3', 
              'bert-base-multilingual-uncased-extremepositive0','bert-base-multilingual-uncased-overandunder3', 
              'bert-base-multilingual-cased4', 'bert-base-multilingual-cased-overandunder1','bert-base-spanish-wwm-uncased4', 
              'bert-base-spanish-wwm-cased4', 'bert-base-spanish-wwm-cased-overandunder2', 'xlm-roberta-base3']
    
    results = pd.DataFrame()
    
    results.insert(loc=0, column = 'tweet_id', value = dev_data['tweet_id'])
    results.insert(loc=1, column = 'True_label', value = dev_data['label'])
    
    for model in models: 
        
        test_print_statement = 'Testing ' + model
        print(test_print_statement)

        model_load_location = '../2022.07.07_task5/saved_models/' + model

        test_result = main(model_load_location)
        results.insert(loc =2, column = model + 'prediction', value =test_result['label'])

    print("Everything successfully completed")


Testing bert-base-multilingual-uncased0
TESTING DATA
Went through 100 steps
TIME TO COMPLETE: 0.49438425302505495

Testing bert-base-multilingual-uncased_oversampled3
TESTING DATA
Went through 100 steps
TIME TO COMPLETE: 0.5109373331069946

Testing bert-base-multilingual-uncased-extremepositive0
TESTING DATA
Went through 100 steps
TIME TO COMPLETE: 0.5148693084716797

Testing bert-base-multilingual-uncased-overandunder3
TESTING DATA
Went through 100 steps
TIME TO COMPLETE: 0.526921010017395

Testing bert-base-multilingual-cased4
TESTING DATA
Went through 100 steps
TIME TO COMPLETE: 0.5409493525822957

Testing bert-base-multilingual-cased-overandunder1
TESTING DATA
Went through 100 steps
TIME TO COMPLETE: 0.5543056766192118

Testing bert-base-spanish-wwm-uncased4
TESTING DATA
Went through 100 steps
TIME TO COMPLETE: 0.5650696714719137

Testing bert-base-spanish-wwm-cased4
TESTING DATA
Went through 100 steps
TIME TO COMPLETE: 0.5710984428723653

Testing bert-base-spanish-wwm-cased-overan

In [78]:
results

,tweet_id,True_label,xlm-roberta-base3prediction,bert-base-spanish-wwm-cased-overandunder2prediction,bert-base-spanish-wwm-cased4prediction,bert-base-spanish-wwm-uncased4prediction,bert-base-multilingual-cased-overandunder1prediction,bert-base-multilingual-cased4prediction,bert-base-multilingual-uncased-overandunder3prediction,bert-base-multilingual-uncased-extremepositive0prediction,bert-base-multilingual-uncased_oversampled3prediction,bert-base-multilingual-uncased0prediction
0,100460,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions
1,301908,non-personal_reports,non-personal_reports,Self_reports,Self_reports,Self_reports,non-personal_reports,Self_reports,Self_reports,Self_reports,Self_reports,Self_reports
2,300889,non-personal_reports,non-personal_reports,non-personal_reports,Lit-News_mentions,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,Lit-News_mentions
3,101501,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions
4,105013,Lit-News_mentions,Lit-News_mentions,non-personal_reports,non-personal_reports,Self_reports,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,Self_reports,Lit-News_mentions
...,...,...,...,...,...,...,...,...,...,...,...,...
2006,301284,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,Lit-News_mentions,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports
2007,100251,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions
2008,103444,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,non-personal_reports,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions
2009,104896,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,non-personal_reports,Lit-News_mentions,Lit-News_mentions,Self_reports,Lit-News_mentions,Lit-News_mentions


In [79]:
results.to_csv('results_Version1.tsv', sep="\t", index = False)

In [80]:
df = pd.read_table("results_Version1.tsv")

In [81]:
df

,tweet_id,True_label,xlm-roberta-base3prediction,bert-base-spanish-wwm-cased-overandunder2prediction,bert-base-spanish-wwm-cased4prediction,bert-base-spanish-wwm-uncased4prediction,bert-base-multilingual-cased-overandunder1prediction,bert-base-multilingual-cased4prediction,bert-base-multilingual-uncased-overandunder3prediction,bert-base-multilingual-uncased-extremepositive0prediction,bert-base-multilingual-uncased_oversampled3prediction,bert-base-multilingual-uncased0prediction
0,100460,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions
1,301908,non-personal_reports,non-personal_reports,Self_reports,Self_reports,Self_reports,non-personal_reports,Self_reports,Self_reports,Self_reports,Self_reports,Self_reports
2,300889,non-personal_reports,non-personal_reports,non-personal_reports,Lit-News_mentions,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,Lit-News_mentions
3,101501,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions
4,105013,Lit-News_mentions,Lit-News_mentions,non-personal_reports,non-personal_reports,Self_reports,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,Self_reports,Lit-News_mentions
...,...,...,...,...,...,...,...,...,...,...,...,...
2006,301284,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,Lit-News_mentions,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports
2007,100251,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions
2008,103444,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,non-personal_reports,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions
2009,104896,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,non-personal_reports,Lit-News_mentions,Lit-News_mentions,Self_reports,Lit-News_mentions,Lit-News_mentions


In [88]:
df2 = df.copy()
df2

,tweet_id,True_label,xlm-roberta-base3prediction,bert-base-spanish-wwm-cased-overandunder2prediction,bert-base-spanish-wwm-cased4prediction,bert-base-spanish-wwm-uncased4prediction,bert-base-multilingual-cased-overandunder1prediction,bert-base-multilingual-cased4prediction,bert-base-multilingual-uncased-overandunder3prediction,bert-base-multilingual-uncased-extremepositive0prediction,bert-base-multilingual-uncased_oversampled3prediction,bert-base-multilingual-uncased0prediction
0,100460,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions
1,301908,non-personal_reports,non-personal_reports,Self_reports,Self_reports,Self_reports,non-personal_reports,Self_reports,Self_reports,Self_reports,Self_reports,Self_reports
2,300889,non-personal_reports,non-personal_reports,non-personal_reports,Lit-News_mentions,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,Lit-News_mentions
3,101501,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions
4,105013,Lit-News_mentions,Lit-News_mentions,non-personal_reports,non-personal_reports,Self_reports,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,Self_reports,Lit-News_mentions
...,...,...,...,...,...,...,...,...,...,...,...,...
2006,301284,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,Lit-News_mentions,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports,non-personal_reports
2007,100251,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions
2008,103444,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,non-personal_reports,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions
2009,104896,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,Lit-News_mentions,non-personal_reports,Lit-News_mentions,Lit-News_mentions,Self_reports,Lit-News_mentions,Lit-News_mentions


In [89]:
df2 = df2.replace(['Lit-News_mentions', 'non-personal_reports','Self_reports'], [0,1,2])
df2

,tweet_id,True_label,xlm-roberta-base3prediction,bert-base-spanish-wwm-cased-overandunder2prediction,bert-base-spanish-wwm-cased4prediction,bert-base-spanish-wwm-uncased4prediction,bert-base-multilingual-cased-overandunder1prediction,bert-base-multilingual-cased4prediction,bert-base-multilingual-uncased-overandunder3prediction,bert-base-multilingual-uncased-extremepositive0prediction,bert-base-multilingual-uncased_oversampled3prediction,bert-base-multilingual-uncased0prediction
0,100460,0,0,0,0,0,0,0,0,0,0,0
1,301908,1,1,2,2,2,1,2,2,2,2,2
2,300889,1,1,1,0,1,1,1,1,1,1,0
3,101501,0,0,0,0,0,0,0,0,0,0,0
4,105013,0,0,1,1,2,1,1,1,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2006,301284,1,1,1,1,0,1,1,1,1,1,1
2007,100251,0,0,0,0,0,0,0,0,0,0,0
2008,103444,0,0,0,0,0,1,0,0,0,0,0
2009,104896,0,0,0,0,0,1,0,0,2,0,0


In [93]:
df2['majority_label'] = df2[df2.columns[2:]].mode(axis=1)[0]

In [94]:
df2

,tweet_id,True_label,xlm-roberta-base3prediction,bert-base-spanish-wwm-cased-overandunder2prediction,bert-base-spanish-wwm-cased4prediction,bert-base-spanish-wwm-uncased4prediction,bert-base-multilingual-cased-overandunder1prediction,bert-base-multilingual-cased4prediction,bert-base-multilingual-uncased-overandunder3prediction,bert-base-multilingual-uncased-extremepositive0prediction,bert-base-multilingual-uncased_oversampled3prediction,bert-base-multilingual-uncased0prediction,majority_label
0,100460,0,0,0,0,0,0,0,0,0,0,0,0.0
1,301908,1,1,2,2,2,1,2,2,2,2,2,2.0
2,300889,1,1,1,0,1,1,1,1,1,1,0,1.0
3,101501,0,0,0,0,0,0,0,0,0,0,0,0.0
4,105013,0,0,1,1,2,1,1,1,1,2,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006,301284,1,1,1,1,0,1,1,1,1,1,1,1.0
2007,100251,0,0,0,0,0,0,0,0,0,0,0,0.0
2008,103444,0,0,0,0,0,1,0,0,0,0,0,0.0
2009,104896,0,0,0,0,0,1,0,0,2,0,0,0.0


In [95]:
df2['majority_label'] = df2['majority_label'].astype(int)

In [96]:
df2

,tweet_id,True_label,xlm-roberta-base3prediction,bert-base-spanish-wwm-cased-overandunder2prediction,bert-base-spanish-wwm-cased4prediction,bert-base-spanish-wwm-uncased4prediction,bert-base-multilingual-cased-overandunder1prediction,bert-base-multilingual-cased4prediction,bert-base-multilingual-uncased-overandunder3prediction,bert-base-multilingual-uncased-extremepositive0prediction,bert-base-multilingual-uncased_oversampled3prediction,bert-base-multilingual-uncased0prediction,majority_label
0,100460,0,0,0,0,0,0,0,0,0,0,0,0
1,301908,1,1,2,2,2,1,2,2,2,2,2,2
2,300889,1,1,1,0,1,1,1,1,1,1,0,1
3,101501,0,0,0,0,0,0,0,0,0,0,0,0
4,105013,0,0,1,1,2,1,1,1,1,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006,301284,1,1,1,1,0,1,1,1,1,1,1,1
2007,100251,0,0,0,0,0,0,0,0,0,0,0,0
2008,103444,0,0,0,0,0,1,0,0,0,0,0,0
2009,104896,0,0,0,0,0,1,0,0,2,0,0,0


In [97]:
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score


In [103]:
majority_accuracy = accuracy_score(df2['True_label'], df2['majority_label'])
print(majority_accuracy)

0.8463451019393337


In [106]:
majority_precision = precision_score(df2['True_label'], df2['majority_label'], labels=[0,1,2], average = None, zero_division=0)[2]
print(majority_precision)

0.670995670995671


In [107]:
majority_recall = recall_score(df2['True_label'], df2['majority_label'], labels=[0,1,2], average = None, zero_division=0)[2]
print(majority_recall)

0.9011627906976745


In [108]:
majority_f1 = f1_score(df2['True_label'], df2['majority_label'], labels=[0,1,2], average = None, zero_division=0)[2]
print(majority_f1)



0.7692307692307693


In [115]:
df2

,tweet_id,True_label,xlm-roberta-base3prediction,bert-base-spanish-wwm-cased-overandunder2prediction,bert-base-spanish-wwm-cased4prediction,bert-base-spanish-wwm-uncased4prediction,bert-base-multilingual-cased-overandunder1prediction,bert-base-multilingual-cased4prediction,bert-base-multilingual-uncased-overandunder3prediction,bert-base-multilingual-uncased-extremepositive0prediction,bert-base-multilingual-uncased_oversampled3prediction,bert-base-multilingual-uncased0prediction,majority_label
0,100460,0,0,0,0,0,0,0,0,0,0,0,0
1,301908,1,1,2,2,2,1,2,2,2,2,2,2
2,300889,1,1,1,0,1,1,1,1,1,1,0,1
3,101501,0,0,0,0,0,0,0,0,0,0,0,0
4,105013,0,0,1,1,2,1,1,1,1,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006,301284,1,1,1,1,0,1,1,1,1,1,1,1
2007,100251,0,0,0,0,0,0,0,0,0,0,0,0
2008,103444,0,0,0,0,0,1,0,0,0,0,0,0
2009,104896,0,0,0,0,0,1,0,0,2,0,0,0


In [122]:
multi = df2.copy()
multi = multi.drop(columns= ['xlm-roberta-base3prediction', 'bert-base-spanish-wwm-cased-overandunder2prediction', 'bert-base-spanish-wwm-cased4prediction', 'bert-base-spanish-wwm-uncased4prediction' ])

In [128]:
multi.columns[-1]

'majority_label'

In [134]:
multi['majority_label_new'] = multi[multi.columns[2:-1]].mode(axis=1)[0].astype(int)

In [138]:
multi = multi.drop(columns = ['majority_label'])

In [139]:
multi

,tweet_id,True_label,bert-base-multilingual-cased-overandunder1prediction,bert-base-multilingual-cased4prediction,bert-base-multilingual-uncased-overandunder3prediction,bert-base-multilingual-uncased-extremepositive0prediction,bert-base-multilingual-uncased_oversampled3prediction,bert-base-multilingual-uncased0prediction,majority_label_new
0,100460,0,0,0,0,0,0,0,0
1,301908,1,1,2,2,2,2,2,2
2,300889,1,1,1,1,1,1,0,1
3,101501,0,0,0,0,0,0,0,0
4,105013,0,1,1,1,1,2,0,1
...,...,...,...,...,...,...,...,...,...
2006,301284,1,1,1,1,1,1,1,1
2007,100251,0,0,0,0,0,0,0,0
2008,103444,0,1,0,0,0,0,0,0
2009,104896,0,1,0,0,2,0,0,0


In [141]:
majority_f1_multi = f1_score(multi['True_label'], multi['majority_label_new'], labels=[0,1,2], average = None, zero_division=0)[2]
print(majority_f1_multi)

0.771356783919598


In [144]:
majority_accuracy_multi = accuracy_score(multi['True_label'], multi['majority_label_new'])
print(majority_accuracy_multi)

0.8378915962207857


In [146]:
df2.to_csv('results_dev_version2.tsv', sep="\t", index = False)
multi.to_csv('results_dev_multi_version1', sep="\t", index = False )